# TODOs

In [ ]:
from IPython.display import display, Markdown

display(Markdown('TODO.md'))

# Design Pattern Recognition with Software Metrics

## Library/Package Imports
All required modules should be in the next cell to avoid scattered imports

In [ ]:
# Ignore missing imports warnings in vs code
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from typing import Callable
import ipywidgets as widgets
from IPython.display import display, HTML
from typing import Optional, Dict

In [ ]:
# Common utility functions
def get_metric_columns():
    return ['NOF', 'NSF', 'NOM', 'NSM', 'NOAM', 'NORM', 'NOPC', 'NOOF', 'NCOF']

def get_ck_metric_columns():
    return ['CBO', 'FAN_IN', 'FAN_OUT']

def get_feature_columns():
    return get_metric_columns() + get_ck_metric_columns() + ['role_kind']

def get_label_column():
    return ['role']

def generate_subplot(df: pd.DataFrame, plot_func: Callable[[pd.DataFrame, str], go.Figure], subplot_width: int = 600, subplot_height: int = 2400) -> go.Figure:
    metric_columns = get_metric_columns() + get_ck_metric_columns()
    subplots = make_subplots(
        len(metric_columns), subplot_titles=metric_columns)
    for i, metric in enumerate(metric_columns):
        figure = plot_func(df, metric)
        subplots.add_trace(figure, row=i+1, col=1)
    subplots['layout'].update(height=subplot_height, width=subplot_width)
    return subplots


def generate_selectable_graph_for_metrics(df: pd.DataFrame, initial_plot_func: Callable[[], go.Figure], update_func: Callable[[go.Figure, pd.DataFrame, str], None], y_label: Optional[str] = None):
    metric_dropdown = widgets.Dropdown(options=get_metric_columns() + get_ck_metric_columns())
    fig = go.FigureWidget(initial_plot_func())

    def on_metric_changed(change):
        metric = change['new']
        with fig.batch_update():
            figure = fig.data[0]
            update_func(figure, df, metric)
            figure['name'] = metric
            label = y_label if y_label else ' '
            fig.update_layout(title=metric, bargap=0.5,
                              xaxis_title=metric, yaxis_title=label)

    metric_dropdown.observe(on_metric_changed, names='value')
    display(widgets.VBox([metric_dropdown, fig]))


def filter_extended_projects(df: pd.DataFrame):
    return df[~df['project'].str.contains('DrJava')]

## Generation of metrics

If the metrics are not yet generated, the following steps are required:

1. Make sure that `source_files.zip` is located in the current directory. The archive contains the actual zipped source code of the projects in [P-MArT](https://www.ptidej.net/tools/designpatterns/) and `pmart.xml` with descriptions of the micro architectures
2. Create a new virtual Python environment with `python -m venv .` in the current directory if not yet done
3. Activate the virtual environment ([refer here for the actual command to run](https://docs.python.org/3/library/venv.html#how-venvs-work))
4. Execute `python3 preprocess_source_files.py` to extract the source files from `source_files.zip` and move the source files described in `pmart.xml` into `dataset` directory. For more information run `python3 preprocess_source_files.py -h`.
    - Source files are structured as `<dataset_dir>/<design_pattern/micro_architecture_<id>`
    - Each micro architecture directory contains the following files:
        - `roles.csv`: Roles, entity names and role kind as described in `pmart.xml`
        - `projects.txt`: From which project the source files come from
        - The source files to be evaluated
5. 
    - **OLD**: Execute `python3 generate_source_file_metrics.py` to generate `metrics.csv`. For more information run `python3 generate_source_file_metrics.py`.
    - **NEW**: Execute `docker build --file docker/sourcefileparser.dockerfile . -t sourcefilerparser:latest` in the `project` directory to build the tool and run `docker run -v ./:/home/app/volume  -e DATASET_PATH=./dataset -e OUTPUT_CSV=./m.csv sourcefilerparser:latest` for metric generation

**NOTES**: 
- As the projects in this dataset are old and not all projects listed in P-MaRT are not accessible, some source files and their entries in `metrics` may be missing.
- The tool for generating the metrics was originally written with a Java Parser implemented Python only. This lead to parsing issues in some source files. As a result, the tool was rewritten as a Java project with a native parser. The original Python script is included for completeness.

## Overview about `metrics.csv`

In order to detect applied Gang Of Four design patterns in source code with machine learning strategies, we first need to transform the source file into a numerical representation that can be understood by a machine learning model.
This approach aims to solve this by generating numerical characteristics for each source file in the context of the regarded micro architecture. As there are several methods to define what metrics to include in the evaluation, the metrics as described [in this paper](../sources/JSEA-DP-2014.pdf):

- NOF: Number of fields
- NSF: Number of static fields
- NOM: Number of methods
- NSM: Number of static methods
- NOAM: Number of abstract methods
- NORM: Number of overridden methods
- NOPC: Number of private constrcutors
- NOOF: Number of object fields
- NCOF: Number of other classes with field of own type


In addition to these metrics, the following Chidamber & Kemerer object-oriented metrics were added to quantify the relation, coupling and cohesion between participants in a design pattern:

- FAN_IN: Number of input dependencies
- FAN_OUT: Number of output dependencies
- CBO: Coupling between objects

## Outlier Detection and Removal

As the dataset may contain a varied implementation of datasets, outlier detection and removal may be required to reduce the noise in the dataset. `sklearn` provides the some automatic and unsupervised approaches out of the box. The following are considered

**NOTE**: This list is subject to change

* Isolation Forest
* Local Outlier Factor

In [ ]:
# Required imports for this section
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

### Isolation Forest

In [ ]:
def apply_isolation_forest(df: pd.DataFrame):
    df_filtered = df.copy()
    isolation_forest = IsolationForest(contamination=0.1)
    df_filtered['outlier'] = isolation_forest.fit_predict(
        df_filtered[get_metric_columns()])
    df_filtered = df_filtered[df_filtered['outlier'] == 1]
    return df_filtered.drop(columns=['outlier'])

### Isolation Forest

In [ ]:
def apply_local_outlier_factor(df: pd.DataFrame) -> pd.DataFrame:
    threshold = 0

    df_copy = df.copy()
    lof = LocalOutlierFactor(contamination=0.5)
    df_copy['outlier_score'] = lof.fit_predict(df_copy[get_metric_columns()])
    df_copy = df_copy[df_copy['outlier_score'] > threshold]
    return df_copy.drop(columns=('outlier_score'))

## Explorative Data Analysis of the Dataset

In [ ]:
df = pd.read_csv('./metrics_ck.csv')
df = filter_extended_projects(df)
df = apply_isolation_forest(df)
print(f'{df.shape[0]} rows were imported')

In [ ]:
df['role'] = df['role'].str.lower().str.strip()
df['role_kind'] = df['role_kind'].str.lower().str.strip()

In [ ]:
# Check if columns in dataframe have expected types
df.dtypes

### Filter Dataframe entries by micro architecture

In [ ]:
micro_arches = df['micro_architecture'].unique().tolist()


def view(micro_arch=''):
    display(df[df['micro_architecture'] == micro_arch]
            [['role', 'role_kind', 'entity'] + get_metric_columns() + get_ck_metric_columns()], clear=True)


w = widgets.Dropdown(options=micro_arches)
widgets.interactive(view, micro_arch=w)

### Distribution of roles

In [ ]:
px.histogram(df, x='role', histfunc='count')

### Distribution of design patterns

In [ ]:
df_binned_by_micro_arch_dp = df.copy()
df_binned_by_micro_arch_dp = df_binned_by_micro_arch_dp.drop_duplicates(
    ['micro_architecture', 'design_pattern']).reset_index()
df_binned_by_micro_arch_dp = df_binned_by_micro_arch_dp['design_pattern'].value_counts(
).reset_index()

fig = px.histogram(df_binned_by_micro_arch_dp, x='design_pattern', y='count')
fig.update_layout(xaxis_title='Design Pattern',
                  yaxis_title='Count in Design Pattern')

### Distribution for metrics

In [ ]:
def initial_histogram():
    return go.Histogram(
        histfunc='count',
    )


def update_histogram(figure: go.Figure, df: pd.DataFrame, metric: str):
    figure['x'] = df[metric]


generate_selectable_graph_for_metrics(
    df, initial_histogram, update_histogram, 'count')

### Box Plots for metrics

In [ ]:
def initial_histogram():
    return go.Box(
    )


def update_histogram(figure: go.Figure, df: pd.DataFrame, metric: str):
    figure['x'] = df[metric]


generate_selectable_graph_for_metrics(df, initial_histogram, update_histogram)

## Model Training

As design patterns can be considered as small scale appliances of software architecture, they consist of different entities with different relationships and roles to fulfill in the regarded design pattern. In order to detect design patterns, we first need to detect what kind of role a given Java class or entity it most likely corresponds to. To achieve this, machine learning model capable of classifying multiple labels should be considered. The extracted software metrics are the numerical inputs and the most likely roles in a design pattern are the result. 
As this falls in the area of supervised machine learning, initially the following models/techniques are to be considered:

**NOTE:** This list is subject to change 

* Support Vector Machines
* Tree Classifiers
* Ensemble Classifiers (e.g Random Forest Classifier)
* Custom Convoluted Network

In order to optimize the given results of a given model, first RandomGridSearch is applied to determine a range of values or selection for the hyperparameters while GridSearch is used to determine the most optimal available value or selection for the regarded hyperparameter.

In [ ]:
# Required import for machine learning
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import hpsklearn
import hyperopt

In [ ]:
# Shared functions in this section
def label_encoder():
    label_encoder = LabelEncoder()

    def encode(col: pd.Series):
        return label_encoder.fit_transform(col)

    def decode(encoded_col: pd.Series):
        return label_encoder.inverse_transform(encoded_col)

    return encode, decode


def preprocess(df: pd.DataFrame):
    df_copy = df.copy()
    role_kind_encode, role_kind_decode = label_encoder()
    role_encode, role_decode = label_encoder()
    df_copy['role_kind'] = role_kind_encode(df_copy['role_kind'])
    df_copy['role'] = role_encode(df['role'])
    train, test = train_test_split(
        df_copy[get_feature_columns() + get_label_column()], test_size=0.25)
    return (train, test, {'role_kind': role_kind_decode, 'role': role_decode})

### Prepare Datset

In [ ]:
#Limit design patterns to Top 3 with most samples
df_dataset = df[(df['design_pattern'] == 'adapter') | (df['design_pattern'] == 'observer') | (df['design_pattern'] == 'abstract_factory')]
df_train, df_test, decoder = preprocess(df_dataset)

### Support Vector Machines

In [ ]:
def apply_svm(df_train: pd.DataFrame, df_test: pd.DataFrame):
    X_train = df_train[get_feature_columns()]
    y_train = df_train[get_label_column()].values.ravel()

    X_test = df_test[get_feature_columns()]
    y_test = df_test[get_label_column()].values.ravel()

    standard_scaler = StandardScaler()
    X_train = standard_scaler.fit_transform(X_train)
    X_test = standard_scaler.fit_transform(X_test)

    svm_classifier = svm.SVC(kernel='rbf', gamma=0.1, C=1.75)
    svm_classifier.fit(X_train, y_train)

    pred = svm_classifier.predict(X_test)
    # print(f1_score(y_true=y_test, y_pred=pred, average='weighted'))
    return svm_classifier.score(X_test, y_test)


apply_svm(df_train, df_test)

### Random Forest Classifier

In [ ]:
def apply_random_forest(df_train: pd.DataFrame, df_test: pd.DataFrame):
    X_train = df_train[get_feature_columns()]
    y_train = df_train[get_label_column()].values.ravel()

    X_test = df_test[get_feature_columns()]
    y_test = df_test[get_label_column()].values.ravel()

    random_forest_classifier = RandomForestClassifier(
        max_depth=30, random_state=1)
    random_forest_classifier.fit(X_train, y_train)

    pred = random_forest_classifier.predict(X_test)
    # print(f1_score(y_true=y_test, y_pred=pred, average='weighted'))
    return random_forest_classifier.score(X_test, y_test)


apply_random_forest(df_train, df_test)

### Get Best Possible Classifier with hyperopt-sklearn

In [ ]:
def apply_hyperopt(df_train: pd.DataFrame, df_test: pd.DataFrame):
    X_train = df_train[get_feature_columns()]
    y_train = df_train[get_label_column()].values.ravel()

    X_test = df_test[get_feature_columns()]
    y_test = df_test[get_label_column()].values.ravel()
    chosen_classifiers = [
        #TODO
    ]

    p = 1 / len(chosen_classifiers)
    classifiers = hp.pchoice('cls', [(p, c) for c in chosen_classifiers])

    hyper_estimator = hpsklearn.HyperoptEstimator(
        classifier=classifiers,
        preprocessing=[],
        max_evals=20,
        algo=hyperopt.tpe.suggest,
        trial_timeout=300,
    )

    hyper_estimator.fit(X_train, y_train)
    print(hyper_estimator.best_model())
    return hyper_estimator.score(X_test, y_test)

apply_hyperopt(df_train, df_test)
    